In [1]:
from time import sleep
from datetime import datetime

from vnpy.app.script_trader import init_cli_trading
from vnpy.gateway.ctp import CtpGateway
from vnpy.trader.constant import Direction
from vnpy.trader.utility import load_json

In [2]:
# 连接到服务器
setting = load_json("connect_ctp.json")
engine = init_cli_trading([CtpGateway])
engine.connect_gateway(setting, "CTP")

2020-03-31 14:24:05.481235	交易服务器连接成功
2020-03-31 14:24:05.488080	行情服务器连接成功
2020-03-31 14:24:05.542767	交易服务器授权验证成功
2020-03-31 14:24:05.556473	行情服务器登录成功
2020-03-31 14:24:05.573112	交易服务器登录成功
2020-03-31 14:24:05.634546	结算信息确认成功
2020-03-31 14:24:15.930016	合约信息查询成功


In [3]:
# 设置参数
entry_level = 5
strike = 3700
trading_size = 1
price_add = 1

# 生成合约代码
call_symbol = f"IO2004-C-{strike}.CFFEX"
put_symbol = f"IO2004-P-{strike}.CFFEX"
futures_symbol = "IF2004.CFFEX"
vt_symbols = [call_symbol, put_symbol, futures_symbol]

# 初始化变量
pos_data = {}
target_data = {}

for vt_symbol in vt_symbols:
    pos_data[vt_symbol] = 0
    target_data[vt_symbol] = 0

# 查询合约数据
for vt_symbol in vt_symbols:
    contract = engine.get_contract(vt_symbol)
    print(contract)

ContractData(gateway_name='CTP', symbol='IO2004-C-3700', exchange=<Exchange.CFFEX: 'CFFEX'>, name='IO2004-C-3700', product=<Product.OPTION: '期权'>, size=100, pricetick=0.2, min_volume=1, stop_supported=False, net_position=False, history_data=False, option_strike=3700.0, option_underlying='IO2004', option_type=<OptionType.CALL: '看涨期权'>, option_expiry=datetime.datetime(2020, 4, 17, 0, 0), option_portfolio='IO', option_index='3700.0')
ContractData(gateway_name='CTP', symbol='IO2004-P-3700', exchange=<Exchange.CFFEX: 'CFFEX'>, name='IO2004-P-3700', product=<Product.OPTION: '期权'>, size=100, pricetick=0.2, min_volume=1, stop_supported=False, net_position=False, history_data=False, option_strike=3700.0, option_underlying='IO2004', option_type=<OptionType.PUT: '看跌期权'>, option_expiry=datetime.datetime(2020, 4, 17, 0, 0), option_portfolio='IO', option_index='3700.0')
ContractData(gateway_name='CTP', symbol='IF2004', exchange=<Exchange.CFFEX: 'CFFEX'>, name='股指2004', product=<Product.FUTURES: '期货'

In [4]:
# 订阅行情
engine.subscribe(vt_symbols)

In [5]:
# 定义持仓查询函数
def get_net_pos(engine, vt_symbol):
    net_pos = 0

    long_position = engine.get_position(vt_symbol + "." + Direction.LONG.value)
    if long_position:
        net_pos += long_position.volume

    short_position = engine.get_position(vt_symbol + "." + Direction.SHORT.value)
    if short_position:
        net_pos -= short_position.volume

    return net_pos

In [6]:
# 循环执行
while True:
    # 获取行情
    call_tick = engine.get_tick(call_symbol)
    put_tick = engine.get_tick(put_symbol)
    futures_tick = engine.get_tick(futures_symbol)

    synthetic_bid = call_tick.bid_price_1 - put_tick.ask_price_1 + strike
    synthetic_ask = call_tick.ask_price_1 - put_tick.bid_price_1 + strike

    futures_bid = futures_tick.bid_price_1
    futures_ask = futures_tick.ask_price_1 

    print(str(datetime.now()))
    print(f"合成期货，买价：{synthetic_bid}，卖价：{synthetic_ask}")
    print(f"标的期货，买价：{futures_bid}，卖价：{futures_ask}")
    
    # 计算交易目标
    # 开仓
    if not target_data[futures_symbol]:
        # 合成高于标的
        if synthetic_bid >= futures_ask + entry_level: 
            print("满足开仓条件，卖看涨，买看跌，买期货")
            target_data[call_symbol] = -3 * trading_size
            target_data[put_symbol] = 3 * trading_size
            target_data[futures_symbol] = 1 * trading_size
        
        # 合成低于标的
        if synthetic_ask <= futures_bid - entry_level:
            print("满足开仓条件，买看涨，卖看跌，卖期货")
            target_data[call_symbol] = 3 * trading_size
            target_data[put_symbol] = -3 * trading_size
            target_data[futures_symbol] = -1 * trading_size
    # 平仓
    else:
        # 
        if (target_data[futures_symbol] > 0 and synthetic_ask <= futures_bid):
            print("满足平仓条件")
            target_data[call_symbol] = 0
            target_data[put_symbol] = 0
            target_data[futures_symbol] = 0
        
        if (target_data[futures_symbol] < 0 and synthetic_bid >= futures_ask):
            print("满足平仓条件")
            target_data[call_symbol] = 0
            target_data[put_symbol] = 0
            target_data[futures_symbol] = 0
    
    # 检查委托情况
    active_orders = engine.get_all_active_orders()
    if active_orders:
        print("当前存在活动委托，执行撤单")
        for order in active_orders:
            engine.cancel_order(order.vt_orderid)
        
        continue
    
    # 查询持仓
    pos_data[call_symbol] = get_net_pos(engine, call_symbol)
    pos_data[put_symbol] = get_net_pos(engine, put_symbol)
    pos_data[futures_symbol] = get_net_pos(engine, futures_symbol)

    print(f"看涨期权{call_symbol}持仓：{pos_data[call_symbol]}")
    print(f"看跌期权{put_symbol}持仓：{pos_data[put_symbol]}")
    print(f"期货期权{futures_symbol}持仓：{pos_data[futures_symbol]}")
    
    # 执行交易
    for vt_symbol in vt_symbols:
        pos = pos_data[vt_symbol]
        target = target_data[vt_symbol]
        diff = target - pos
        
        # 持仓和目标一致，无需交易
        if not diff:
            continue
        # 大于则买入
        elif diff > 0:
            # 有空头持仓，买入平仓
            if pos < 0:
                # self.cover(vt_symbol, tick.ask_price_1 + price_add, abs(diff))
                print(f"cover {vt_symbol}")
            # 否则买入开仓
            else:
                # self.buy(vt_symbol, tick.ask_price_1 + price_add, abs(diff))
                print(f"buy {vt_symbol}")
        # 小于则卖出
        elif diff < 0:
            # 有多头持仓，卖出平仓
            if pos > 0:
                # self.sell(vt_symbol, tick.bid_price_1 - price_add, abs(diff))
                print(f"sell {vt_symbol}")
            # 否则卖出开仓
            else:
                # self.short(vt_symbol, tick.bid_price_1 - price_add, abs(diff))
                print(f"short {vt_symbol}")
        
    # 等待下一轮
    sleep(30)


2020-03-31 14:37:24.750425
合成期货，买价：3669.8，卖价：3671.2
标的期货，买价：3669.6，卖价：3670.2
看涨期权IO2004-C-3700.CFFEX持仓：0
看跌期权IO2004-P-3700.CFFEX持仓：0
期货期权IF2004.CFFEX持仓：0
2020-03-31 14:37:54.751370
合成期货，买价：3668.2，卖价：3670.2
标的期货，买价：3668.2，卖价：3669.0
看涨期权IO2004-C-3700.CFFEX持仓：0
看跌期权IO2004-P-3700.CFFEX持仓：0
期货期权IF2004.CFFEX持仓：0
2020-03-31 14:38:24.754385
合成期货，买价：3669.2，卖价：3670.6
标的期货，买价：3668.6，卖价：3669.4
看涨期权IO2004-C-3700.CFFEX持仓：0
看跌期权IO2004-P-3700.CFFEX持仓：0
期货期权IF2004.CFFEX持仓：0


KeyboardInterrupt: 